# Assignment 1: Predict CPU Efficiency of HPC Cluster Jobs

In a HPC environment, users submit their scripts/programs as jobs to a Job Scheduler. 
Users can request for resources for their scripts/programs, e.g.: Number of CPU cores, Number of GPUs, Amount of Memory (RAM).
The Job Scheduler then dispatches the user's job to remote servers (nodes) when resources are available.

The provided dataset consists of logs of completed/ended jobs. It records the requested resource values as well as actual resource consumption values of each job.

The task for this assignment is to predict the job's CPU utilisation efficiency.
An ideal efficient job should use as close as possible to 100%, however realistically we define efficient CPU utilisation as above 75% of requested CPU cores.

**CPU_EFF = (USED_CPUTIME / USED_WALLTIME) / REQUESTED_NCPUS**


## Data Description

### Columns
'user','dept','queue','Exit_status',
'Resource_List.fluent_lic', 'Resource_List.mem',
'Resource_List.mpiprocs', 'Resource_List.ncpus', 'Resource_List.ngpus',
'Resource_List.nodect', 'Resource_List.walltime', 'cluster', 'ctime', 'datetime', 'end',
'etime', 'exec_host', 'exec_vnode', 'group', 'job_id',
'qtime', 'resources_used.GPU_duration','resources_used.GPU_maxGpuMemoryUsed',
'resources_used.GPU_energyConsumed', 'resources_used.cpupercent',
'resources_used.cput', 'resources_used.mem', 'resources_used.ncpus',
'resources_used.vmem', 'resources_used.walltime',
'run_count', 'start', 'status', 'wait_time', 'gpu0.mem_max', 'gpu0.smUtil_avg',
'gpu0.smUtil_max', 'gpu1.mem_max', 'gpu1.smUtil_avg', 'gpu1.smUtil_max',
'gpu2.mem_max', 'gpu2.smUtil_avg', 'gpu2.smUtil_max', 'gpu3.mem_max',
'gpu3.smUtil_avg', 'gpu3.smUtil_max',


## Load Data

## EDA

## Data Preprocessing

## Feature Selection/Engineering

## Model Development

## Train

## Evaluation

Evaluate your model's performance using relevant metrics

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

df= pd.read_csv("filtered_062019.csv",index_col=0)
df1=df.dropna()
df1 = df1.drop_duplicates()
df1= df1.drop(['gpu0.mem_max', 'gpu0.smUtil_avg', 'gpu0.smUtil_max', 'gpu1.mem_max', 'gpu1.smUtil_avg', 'gpu1.smUtil_max', 'gpu2.mem_max', 'gpu2.smUtil_avg', 'gpu2.smUtil_max', 'gpu3.mem_max', 'gpu3.smUtil_avg', 'gpu3.smUtil_max'],axis=1)
df1=df1.drop(['run_count', 'start', 'status', 'wait_time','cluster', 'ctime', 'datetime', 'end', 'etime', 'exec_host', 'exec_vnode', 'group', 'job_id', 'qtime'],axis=1)
df1=df1.drop(['Exit_status', 'Resource_List.fluent_lic','Resource_List.mpiprocs','Resource_List.ngpus','resources_used.GPU_duration','resources_used.GPU_maxGpuMemoryUsed', 'resources_used.GPU_energyConsumed','Resource_List.nodect'],axis=1)
df1['CPU_eff']= (df1['resources_used.cput']/df1['resources_used.walltime'])/df1['Resource_List.ncpus']
df1 = df1.replace([np.inf, -np.inf], np.nan)
df1=df1.dropna()
X = df1.drop('CPU_eff', axis=1)
y = df1['CPU_eff']

numeric_features = ['Resource_List.mem','Resource_List.ncpus','Resource_List.walltime','resources_used.cpupercent', 'resources_used.cput','resources_used.mem', 'resources_used.ncpus', 'resources_used.vmem','resources_used.walltime']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=False))])

categorical_features = ['user','dept','queue']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[('preprocess', preprocessor),
                      ('classifier',  RandomForestClassifier(max_depth=2,random_state=0))])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)

lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_test_encoded = lab_enc.fit_transform(y_test)

clf.fit(X_train, y_train_encoded)
print("model score: %.3f" % clf.score(X_test, y_test_encoded))

#rs = np.random.RandomState(0)
#corr = df1.corr()
#corr.style.background_gradient(cmap='coolwarm')

MemoryError: Unable to allocate 35.3 GiB for an array with shape (157539, 30092) and data type float64